## Block Timestamps
**[Johnnatan Messias](https://johnnatan-messias.github.io/), August 2024**

Knowing when a vote was cast is crucial for understanding the dynamics of governance in decentralized autonomous organizations (DAOs). However, the logs for events do not include timestamps, so we need to obtain them separately. To do this, we must gather the block information for every transaction associated with the governance events. Since retrieving this data from an Ethereum Archive node could be time-consuming, we've opted to use the [Nansen Query API](https://query.nansen.ai/) to fetch the block timestamps instead.

This notebook file contains the code used to fetch the block timestamps for the governance events. The timestamps are then saved in a new file named `./data/blocks/block_timestamp_9600000_20563001.csv.gz`.

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import pandas_gbq
from google.cloud import bigquery
%load_ext google.cloud.bigquery


In [2]:
progress_bar_type = 'tqdm_notebook'
project_id = 'Add your project id here'

In [3]:
import os
data_dir = os.path.realpath(os.path.join(
    os.getcwd(), "..", "data", "blocks")) + os.sep
os.makedirs(data_dir, exist_ok=True)

In [4]:
sql = """
SELECT
    number, timestamp
FROM
    `nansen-query.raw_ethereum.blocks`
WHERE
    number >= {block_number_start} AND number <= {block_number_end}
ORDER BY
    number DESC
"""

In [ ]:
block_number_start, block_number_end = 9_600_000, 20_563_001

query = sql.format(block_number_start=block_number_start,
                   block_number_end=block_number_end)

df = pandas_gbq.read_gbq(
    query, project_id=project_id, progress_bar_type=progress_bar_type)

filename = "block_timestamp_{}_{}.csv.gz".format(
    block_number_start, block_number_end)

df.to_csv(data_dir+filename, index=False, compression='gzip')